In [1]:
using CSV
using DataFrames
using Dates
using Distributions
using Serialization
using LinearAlgebra: diagm
using JSON

In [2]:
projectbasepath = "../../covid-resource-allocation/"
include(joinpath(projectbasepath, "src/processing/LocalDataCommon.jl"));
include(joinpath(projectbasepath, "src/processing/GeographicData.jl"));

In [3]:
data_dir = "../data/";
outputdatapath = "../data/";

In [4]:
data_date = "2020_12_10";

In [5]:
capacity_data = DataFrame(CSV.File("../data/capacity_maryland.csv"));

In [6]:
SCENARIOS = [:moderate, :pessimistic, :catastrophic];
BEDTYPES  = [:allbeds, :icu, :acute];

In [7]:
los_dist = (
    icu = Weibull(1.58, 13.32),
    acute = Weibull(1.38, 12.88),
    allbeds = Weibull(1.38, 12.88),
);

In [8]:
start_date = Date(2020, 11, 25);
end_date   = Date(2021, 2, 10);
date_range = collect(start_date : Day(1) : end_date);

In [9]:
hospitals = sort(capacity_data.hospital);
N = length(hospitals);

In [10]:
adj = BitArray(ones(N,N) - diagm(ones(N)));

In [11]:
dist_matrix = diagm(fill(Inf, N));

In [12]:
capacity_names_full = ["Base Capacity"];
capacity_names_abbrev = ["baselinecap"];

In [13]:
function load_capacity_md(hospitals, bedtype, capacity_levels=[:baseline])
    beds_dict = Dict(row.hospital => Dict(
        "icu" => row.capacity_icu,
        "acute" => row.capacity_acute,
        "allbeds" => row.capacity_icu + row.capacity_acute,
    ) for row in eachrow(capacity_data))

    if capacity_levels isa Symbol
        capacity = [beds_dict[h][string(bedtype)] for h in hospitals]
    elseif capacity_levels isa AbstractArray
        capacity = hcat([[beds_dict[h][string(bedtype)] for h in hospitals] for l in capacity_levels]...)
    else
        error("Invalid capacity_levels")
    end

    return capacity
end;

In [14]:
function load_forecast_maryland(scenario, bedtype)
    @assert(bedtype in [:icu, :acute, :allbeds])
    @assert(scenario in [:optimistic, :moderate, :pessimistic, :catastrophic])
    
    date_dir = "forecasts-" * replace(string(data_date), "_" => "-")

    forecast = DataFrame(CSV.File(joinpath(data_dir, "forecast_fake_$(data_date).csv")))
    forecast_dict = Dict((row.hospital, row.date) => row for row in eachrow(forecast))
    
    day0 = minimum(date_range) - Day(1)
    forecast_admitted = [forecast_dict[(h,d)]["admitted_$(bedtype)"] for h in hospitals, d in date_range]
    forecast_active   = [forecast_dict[(h,d)]["admitted_$(bedtype)"] for h in hospitals, d in date_range]

    forecast_admitted_uncertainty = 0.1 .* forecast_admitted
    
    beds = load_capacity_md(hospitals, bedtype, :baseline)
    capacity = load_capacity_md(hospitals, bedtype, [:baseline,])

    data = (
        scenario = scenario,
        bedtype = bedtype,

        los_dist = los_dist[bedtype],

        active = forecast_active,
        admitted = forecast_admitted,
        admitted_uncertainty = forecast_admitted_uncertainty,

        beds = beds,
        capacity = capacity,
    )

    return data
end;

In [15]:
maindata = Dict()
for scenario in SCENARIOS, bedtype in BEDTYPES
    maindata[(scenario,bedtype)] = load_forecast_maryland(scenario, bedtype)
end

In [16]:
hospital_positions_raw = JSON.parsefile("../data/md_hospital_locs.json");
hospital_positions = Dict(h => (
    lat  = hospital_positions_raw[h]["lat"],
    long = hospital_positions_raw[h]["lng"],
) for h in hospitals);

In [17]:
completedata = (
    location_names = hospitals,
    location_names_short = hospitals,
    start_date = start_date,
    end_date = end_date,
    counties = nothing,
    states = nothing,
    dist_matrix = dist_matrix,
    locations_latlong = hospital_positions,
    casesdata = maindata,
);

In [18]:
serialize(joinpath(outputdatapath, "data_maryland.jlser"), completedata);